# Exporting graph as shape, nodes and edges as list

### The file formats to write and export graphs are .shp and .dat. and .pickle
To export the graphs itself in other file formats we suggest the use of networkx functions.
https://networkx.org/documentation/stable/reference/readwrite/index.html

## Load packages

In [1]:
import geopandas as gpd
import networkx as nx
from shapely.geometry import LineString, Point
import pickle

## Reading file

In [2]:
#Read geodataframe with geopandas
name = 'tsan_fractures'
gdf = gpd.read_file(f'data/shapefile/{name}.shp')

In [3]:
#Create graph from shapefile
G = nx.Graph()
coord_to_label = {}
precision = 5 #decimals

for index, row in gdf.iterrows():
    line = row['geometry']
    
    if line is not None:
        coordinates = [(round(coord[0], precision), round(coord[1], precision)) for coord in line.coords]

        for i, coord in enumerate(coordinates):
            if coord not in coord_to_label:
                label_index = len(coord_to_label)
                coord_to_label[coord] = label_index
                G.add_node(label_index, pos=coord)

        edges = [(coord_to_label[coordinates[i]], coord_to_label[coordinates[i + 1]]) for i in range(len(coordinates) - 1)]
        G.add_edges_from(edges)

## Exporting graph

### Shapefile (.shp)

In [4]:
#Export graph as shapefile with a coordinate reference system
crs = 'EPSG:2056'

#Get positions and build geometries for geodataframe
positions = nx.get_node_attributes(G, 'pos')

node_data = {'id': list(positions.keys()), 'geometry': [Point(pos) for pos in positions.values()]}
node_gdf = gpd.GeoDataFrame(node_data, crs=crs)

edge_data = []
for u, v in G.edges():
    edge_data.append({'geometry': LineString([positions[u], positions[v]]), 'source': u, 'target': v})
edge_gdf = gpd.GeoDataFrame(edge_data, crs=crs)

node_gdf.to_file(f'data/output/{name}_node.shp')
edge_gdf.to_file(f'data/output/{name}_edge.shp')

### Edge list

In [5]:
#Export adjacency List
nx.write_edgelist(G, f'data/output/{name}_edgelist')

### Karstnet format (.dat)

In [6]:
#Export edges as list
with open(f'data/output/{name}_edgelist', 'r') as infile, open(f'data/output/{name}_edge.dat', 'w') as outfile:
    for line in infile:
        parts = line.split()
        new_line = f"{int(parts[0]) + 1} {int(parts[1]) + 1}\n"
    
        outfile.write(new_line)

In [7]:
pos_dict = {i: p for i, p in nx.get_node_attributes(G, 'pos').items()}

In [8]:
#Export nodes coordinates as a list
positions = nx.get_node_attributes(G, 'pos')
coordinates_list = list(positions.values())

with open(f'data/output/{name}_nodes.dat', 'w') as outfile:
    for coordinate in coordinates_list:
        outfile.write(f"{coordinate[0]} {coordinate[1]} {0}\n")

### Python object serialization (.pickle)
We suggest python only for exporting and importing from one example to the other in sequence, but not for keeping the file in an archive.

In [10]:
#Export graph as pickle
output = f'data/graphs/{name}.pickle'
pickle.dump(G, open(output, 'wb'))